# Percolation

In [1]:
import collections
import random
import time
from typing import Union

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np

# use the searborn style
plt.style.use("seaborn")
# make interactive plots
plt.ion()

%matplotlib qt

In [2]:
# define the number of points in x and y direction
nx, ny = 10, 10
# define the offset
offset = 1
# define the distance between the nodes
distance = 2
# define the x and y coordinates
x_coord = np.arange(start=offset, stop=nx + offset, step=distance, dtype=float)
y_coord = np.arange(start=offset, stop=ny + offset, step=distance, dtype=float)
# create a mesh
x, y = np.meshgrid(x_coord, y_coord)
# create an array containing the coordinates of each point and the corresponding indices as array
coords = np.array(list(zip(x.flatten(), y.flatten())), dtype=float)
idcs = np.arange(0, coords.shape[0])
# define the search tree as set from which each element which is visited will be removed
tree = {(x, y) for x, y in coords}

# select random coordinates which represent the infected persons
rng = np.random.default_rng()
num_infected = 1
idx = rng.choice(idcs, size=num_infected)
# get a random coordinate which represents the patient zero
pat_zero = tuple(coords[idx, ...][0])
# reshape the indices in order to be consistent with the grid
idcs = np.flip(np.flip(idcs, axis=0).reshape(x.shape), axis=1)

# define the probability of infection
p_infect = 0.9

In [3]:
def get_neighbors(current: tuple, distance: Union[int, float], x_limit: tuple, y_limit: tuple) -> np.ndarray:
    """Get all direct neighbors of an infected person.

    Parameters
    ----------
    current : tuple
        The current node as (x, y) coordinates.
    distance : int or float
        The distance between two nodes.
    x_limit : tuple
        The limit in x direction as (x_min, x_max).
    y_limit : tuple
        The limit in y direction as (y_min, y_max).

    Returns
    -------
    np.ndarray
        The neighboring coordinates as array in the order top, left, bottom, right. If a node does not have
        all neighbors, the corresponding coordinate is skipped in the order, i.e. top, bottom, left if
        the node is at the left end at x_min.

    """

    x, y = current
    x_min, x_max, y_min, y_max = *x_limit, *y_limit
    if x < x_min or x > x_max:
        raise AssertionError("The node is not within the range.")
    if y < y_min or y > y_max:
        raise AssertionError("The node is not within the range.")
    if x == x_min:
        if y == y_min:
            top = np.array([x, y + distance], dtype=float)
            right = np.array([x + distance, y], dtype=float)
            return np.array([top, right])
        elif y == y_max:
            bottom = np.array([x, y - distance], dtype=float)
            right = np.array([x + distance, y], dtype=float)
            return np.array([bottom, right])
        else:
            top = np.array([x, y + distance], dtype=float)
            bottom = np.array([x, y - distance], dtype=float)
            right = np.array([x + distance, y], dtype=float)
            return np.array([top, bottom, right])
    elif x == x_max:
        if y == y_min:
            top = np.array([x, y + distance], dtype=float)
            left = np.array([x - distance, y], dtype=float)
            return np.array([top, left])
        elif y == y_max:
            left = np.array([x - distance, y], dtype=float)
            bottom = np.array([x, y - distance], dtype=float)
            return np.array([left, bottom])
        else:
            top = np.array([x, y + distance], dtype=float)
            left = np.array([x - distance, y], dtype=float)
            bottom = np.array([x, y - distance], dtype=float)
            return np.array([top, left, bottom])
    else:
        if y == y_min:
            top = np.array([x, y + distance], dtype=float)
            left = np.array([x - distance, y], dtype=float)
            right = np.array([x + distance, y], dtype=float)
            return np.array([top, left, right])
        elif y == y_max:
            left = np.array([x - distance, y], dtype=float)
            bottom = np.array([x, y - distance], dtype=float)
            right = np.array([x + distance, y], dtype=float)
            return np.array([left, bottom, right])
        else:
            top = np.array([x, y + distance], dtype=float)
            left = np.array([x - distance, y], dtype=float)
            bottom = np.array([x, y - distance], dtype=float)
            right = np.array([x + distance, y], dtype=float)
            return np.array([top, left, bottom, right])


In [4]:
# DEBUG ################################
tree = {(x, y) for x, y in coords}
counter = 0
tree_struct = {}
#################################

# define the color black with a hex string
black = "#000000"
red = "#ff0000"
# define the figure
nrows, ncols = 1, 1
fig1 = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(nrows, ncols, figure=fig1)
ax1 = fig1.add_subplot(gs[0, 0])
# plot the grid
ax1.plot(x, y, color=black, marker="x", ls="-")
ax1.plot(y, x, color=black, marker="x", ls="-")

# define the x and y limits
xlim, ylim =  (x_coord[0], x_coord[-1]), (y_coord[0], y_coord[-1])
frontier = collections.deque([pat_zero])
ax1.plot(*pat_zero, marker='o', ms=10, color='b', label="patient zero")
while frontier:
    infected = []
    print(f"Frontier: {frontier}")
    node = frontier.pop()
    print(f"Node: {node}")
    # remove the infected patient from the tree
    tree.remove(node)
    neighbors = get_neighbors(node, distance, x_limit=xlim, y_limit=ylim)
    mask = rng.random(size=neighbors.shape[0]) < p_infect
    for x, y in neighbors[mask]: 
        if (x, y) not in tree or (x, y) in frontier:
            continue
        frontier.append((x, y))
        infected.append((x, y))
        ax1.arrow(node[0], node[1], x - node[0], y - node[1], width=0.04, length_includes_head=True, head_width=0.1, color=red)
        ax1.plot(x, y, marker="o", ms=10, color="g")
    tree_struct[f"Node: {node}"] = infected
else:
    print("\n----------------\nEMPTY\n----------------\n")
ax1.legend()

Frontier: deque([(3.0, 7.0)])
Node: (3.0, 7.0)
Frontier: deque([(3.0, 9.0), (3.0, 5.0), (5.0, 7.0)])
Node: (5.0, 7.0)
Frontier: deque([(3.0, 9.0), (3.0, 5.0), (5.0, 5.0), (7.0, 7.0)])
Node: (7.0, 7.0)
Frontier: deque([(3.0, 9.0), (3.0, 5.0), (5.0, 5.0), (7.0, 9.0), (7.0, 5.0), (9.0, 7.0)])
Node: (9.0, 7.0)
Frontier: deque([(3.0, 9.0), (3.0, 5.0), (5.0, 5.0), (7.0, 9.0), (7.0, 5.0), (9.0, 9.0), (9.0, 5.0)])
Node: (9.0, 5.0)
Frontier: deque([(3.0, 9.0), (3.0, 5.0), (5.0, 5.0), (7.0, 9.0), (7.0, 5.0), (9.0, 9.0), (9.0, 3.0)])
Node: (9.0, 3.0)
Frontier: deque([(3.0, 9.0), (3.0, 5.0), (5.0, 5.0), (7.0, 9.0), (7.0, 5.0), (9.0, 9.0), (7.0, 3.0), (9.0, 1.0)])
Node: (9.0, 1.0)
Frontier: deque([(3.0, 9.0), (3.0, 5.0), (5.0, 5.0), (7.0, 9.0), (7.0, 5.0), (9.0, 9.0), (7.0, 3.0), (7.0, 1.0)])
Node: (7.0, 1.0)
Frontier: deque([(3.0, 9.0), (3.0, 5.0), (5.0, 5.0), (7.0, 9.0), (7.0, 5.0), (9.0, 9.0), (7.0, 3.0), (5.0, 1.0)])
Node: (5.0, 1.0)
Frontier: deque([(3.0, 9.0), (3.0, 5.0), (5.0, 5.0), (7.0, 9.

    START Percolation
    
    INIT frontier with patient zero
    INIT tree with patients
    
    WHILE frontier not empty

        INIT node <- frontier.pop
        IF node in frontier THEN
            CONTINUE
        ENDIF
        TRY
            tree.remove(node)
        CATCH NodeNotInTreeError
            CONTINUE
        ENDTRY
        INIT neighbors <- get_neighbors
        INIT infected neighbors <- SUBSET neighbors
        FOR x, y in infected neighbors
            IF x, y not in tree OR x, y in frontier
                CONTINUE
            ENDIF
            frontier.append(x, y)
    

In [6]:
tree

set()